tempfile模块是用来创建临时文件系统对象的。  
- TemporaryFile()打开并返回一个未命名的临时文件
- NamedTemporaryFile()打开并返回一个命名临时文件
- SpooledTemporaryFile()会在将数据写入磁盘之前将内容保存在内存中
- TemporaryDirectory是一个上下文管理器，它会在上下文关闭的时候移除目录

## TemporaryFile临时文件

In [2]:
import os
import tempfile

print('Build a filename with PID:')
filename = f'/tmp/guess_name.{os.getpid()}.txt'
with open(filename, 'w+b') as temp:
    print('temp:')
    print(f"  {temp!r}")
    print('temp.name:')
    print(f"  {temp.name!r}")

# 手动清除临时文件
os.remove(filename)
print()
with tempfile.TemporaryFile() as temp:
    print('temp:')
    print(f"  {temp!r}")
    print('temp.name:')
    print(f"  {temp.name!r}")

Build a filename with PID:
temp:
  <_io.BufferedRandom name='/tmp/guess_name.1819.txt'>
temp.name:
  '/tmp/guess_name.1819.txt'

temp:
  <_io.BufferedRandom name=58>
temp.name:
  58


上例展示了TemporaryFile和普通方式创建临时文件的不同，TemporaryFile创建的文件没有名称。  
TemporaryFile 默认情况下文件描述符使用w+b模式打开，所以它在所有平台上行为一致。

In [3]:
with tempfile.TemporaryFile() as temp:
    temp.write(b"some data")
    temp.seek(0)  # 写入后，将文件描述符内部指针重置到文件开始处以便读回数据
    print(temp.read())

b'some data'


In [5]:
with tempfile.TemporaryFile(mode='w+t') as temp: # w+t表示文本模式
    temp.writelines(["some data", 'some else \n', 'sadsa \n'])
    temp.seek(0)  
    for line in temp:
        print(line.rstrip())

some datasome else
sadsa


## NamedTemporaryFile命名临时文件
对于跨进程甚至跨主机应用程序来说，其各部分之间最简单的数据传递方式是命名文件。NamedTemporaryFile会创建一个文件保留名称，文件在句柄关闭后删除。

In [6]:
import pathlib
with tempfile.NamedTemporaryFile() as temp:
    print('temp:')
    print(f"  {temp!r}")
    print('temp.name:')
    print(f"  {temp.name!r}")
    
    f = pathlib.Path(temp.name)
print(f"Exists after close:{f.exists()}")

temp:
temp.name:
  '/var/folders/r7/mv379gf57vlb8j9mc0nwjb9m0000gn/T/tmpsxrfmryk'
Exists after close:False


## SpooledTemporaryFile缓冲文件
临时文件通常保存小量数据，使用SpooledTemporaryFile可能效率更高，因为它在内容超过max_size之前，使用io.BytesIO或io.StringIO将数据保存在内存中，当内容超过max_size时，数据被写入磁盘保存，同时缓冲池被替换为TemporaryFile。

In [8]:
with tempfile.SpooledTemporaryFile(max_size=100, mode='w+t', encoding='utf-8') as temp:
    print('temp:')
    print(f"  {temp!r}")
    
    for i in range(8):
        temp.write('this line repeat \n')
        print(temp._rolled, temp._file)

temp:
False <_io.StringIO object at 0x10b9a1558>
False <_io.StringIO object at 0x10b9a1558>
False <_io.StringIO object at 0x10b9a1558>
False <_io.StringIO object at 0x10b9a1558>
False <_io.StringIO object at 0x10b9a1558>
True <_io.TextIOWrapper name=58 mode='w+t' encoding='utf-8'>
True <_io.TextIOWrapper name=58 mode='w+t' encoding='utf-8'>
True <_io.TextIOWrapper name=58 mode='w+t' encoding='utf-8'>


In [9]:
with tempfile.SpooledTemporaryFile(max_size=1000, mode='w+t', encoding='utf-8') as temp:
    print('temp:')
    print(f"  {temp!r}")
    
    for i in range(8):
        temp.write('this line repeat \n')
        print(temp._rolled, temp._file)
    temp.rollover()
    print(temp._rolled, temp._file)

temp:
False <_io.StringIO object at 0x10b9a1ca8>
False <_io.StringIO object at 0x10b9a1ca8>
False <_io.StringIO object at 0x10b9a1ca8>
False <_io.StringIO object at 0x10b9a1ca8>
False <_io.StringIO object at 0x10b9a1ca8>
False <_io.StringIO object at 0x10b9a1ca8>
False <_io.StringIO object at 0x10b9a1ca8>
False <_io.StringIO object at 0x10b9a1ca8>
True <_io.TextIOWrapper name=58 mode='w+t' encoding='utf-8'>


上例调整了max_size，除非显式调用rollover()，否则不会创建文件

## TemporaryDirectory 临时目录

In [10]:
with tempfile.TemporaryDirectory() as directory_name:
    the_dir = pathlib.Path(directory_name)
    print(the_dir)
    a_file = the_dir / 'a.txt'
    a_file.write_text('some thing is deleted')
    
print('Dir exists after?', the_dir.exists())
print('Contents after?', list(the_dir.glob('*')))

/var/folders/r7/mv379gf57vlb8j9mc0nwjb9m0000gn/T/tmptr1po___
Dir exists after? False
Contents after? []


## 文件位置和名称

In [11]:
with tempfile.NamedTemporaryFile(suffix='_suf', prefix='pre_', dir='/tmp') as temp:
    print('temp:')
    print(f"  {temp!r}")
    print('temp.name:')
    print(f"  {temp.name!r}")

temp:
temp.name:
  '/tmp/pre_vbqlxyuc_suf'


In [12]:
print('gettempdir: ', tempfile.gettempdir())
print('gettempprefix: ', tempfile.gettempprefix())

gettempdir:  /var/folders/r7/mv379gf57vlb8j9mc0nwjb9m0000gn/T
gettempprefix:  tmp


In [13]:
tempfile.tempdir = '/Home'
print('gettempdir: ', tempfile.gettempdir())

gettempdir:  /Home


gettempdir通过查询一个列表来确定当前进程新创建文件的存放位置，这个列表包括一些环境变量等。